In [7]:
!pip install agno dotenv asyncio openai

In [18]:
import os
import json
import asyncio
from typing import List, Dict, Any
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")
url = "https://foundation-models.api.cloud.ru/v1"

client = OpenAI(
    api_key=api_key,
    base_url=url
)

In [19]:
class Agent:
    def __init__(self, name: str, instructions: str):
        self.name = name
        self.instructions = instructions

    def run(self, previous_speaker: str, topic_summary: str) -> Dict[str, Any]:
        system_prompt = f"""Ты — {self.name}.
{self.instructions}

ТЕКУЩАЯ ТЕМА: Обсуждение Toyota Mark II и почему она стала культовой в России.

ПРАВИЛА:
1. НЕ ПОВТОРЯЙ предыдущие сообщения
2. НЕ ПЕРЕФРАЗИРУЙ то, что уже сказано
3. Добавь НОВУЮ информацию или СВОЙ УНИКАЛЬНЫЙ ВЗГЛЯД
4. Если предыдущий оратор уже что-то обсудил, развивай тему дальше
5. Говори ТОЛЬКО СВОИМИ СЛОВАМИ
6. Не упоминай предыдущего оратора прямо
7. Не используй маркдаун-разметку

Предыдущий участник ({previous_speaker}) уже обсудил: {topic_summary[:200]}...

Теперь твоя очередь высказаться."""

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "Продолжи обсуждение Toyota Mark II. Добавь свою уникальную точку зрения."}
        ]

        response = client.chat.completions.create(
            model="ai-sage/GigaChat3-10B-A1.8B",
            max_tokens=400,
            temperature=0.85,
            top_p=0.9,
            presence_penalty=1.0,
            frequency_penalty=1.2,
            messages=messages
        )

        return {
            "content": response.choices[0].message.content,
            "agent": self.name
        }

In [22]:
def create_agents():
    return [
        Agent(
            name="Денис Михеев",
            instructions=(
                "Твой стиль: аналитичный, эрудированный, увлеченный техническими деталями. "
                "Говори спокойно, четко формулируй мысли, опирайся на исторические факты и технические характеристики. "
                "Расскажи о конкретных технических особенностях Mark II, его эволюции, инженерных решениях. "
                "Упомяни исторический контекст, сравнение с другими моделями Toyota."
            )
        ),
        Agent(
            name="Денис Дорохов",
            instructions=(
                "Твой стиль: энергичный, харизматичный, немного провокационный. "
                "Переключайся между профессиональными темами и популярной культурой. "
                "Используй смелые сравнения и живые метафоры. "
                "Расскажи о тюнинг-культуре, дрифте, о том как Mark II стал частью массового сознания. "
                "Говори о культурном феномене, субкультурах вокруг этой машины."
            )
        ),
        Agent(
            name="Стас Асафьев",
            instructions=(
                "Твой стиль: неформальный, с юмором, иногда с сарказмом. "
                "Выражай мнение рядового автолюбителя, но будь технически подкован. "
                "Вставляй шутки, мемы, отсылки к поп-культуре. "
                "Расскажи о практической стороне владения Mark II, его доступности, "
                "типичных проблемах, реальном опыте владельцев, почему модель стала народной любимицей."
            )
        ),
        Agent(
            name="Дмитрий Шпеньков",
            instructions=(
                "Твой стиль: образный, с элементами стендапа, находи неожиданный ракурс. "
                "Используй иронию, гротеск, преувеличение для создания ярких образов. "
                "Сравни машину с персонажем из фильма, обыграй её социальный статус. "
                "Расскажи историю или анекдот, связанный с Mark II. Будь оригинальным и творческим."
            )
        )
    ]

In [28]:
async def chat(receiver: Agent, previous_speaker: str, topic_summary: str):
    print(f"\n\n{receiver.name}:")

    response = receiver.run(previous_speaker, topic_summary)
    print(f"{response['content']}")
    print("-" * 52)

    return response['content']

In [36]:
async def conversation():
    agents = create_agents()

    print("Беседа экспертов о Toyota Mark II")
    print("=" * 50)

    print("\nТема: Toyota Mark II - почему культовая машина в России?\n")

    print(f"\n{agents[0].name}:")
    first_text = agents[0].run("вступление", "начало обсуждения")
    print(first_text['content'])
    print("-" * 50)

    print(f"\n{agents[1].name}:")
    second_text = agents[1].run(agents[0].name, "начал обсуждение")
    print(second_text['content'])
    print("-" * 50)

    print(f"\n{agents[2].name}:")
    third_text = agents[2].run(agents[1].name, "продолжил обсуждение")
    print(third_text['content'])
    print("-" * 50)

    print(f"\n{agents[3].name}:")
    fourth_text = agents[3].run(agents[2].name, "добавил свои мысли")
    print(fourth_text['content'])
    print("-" * 50)

    print(f"\n{agents[0].name}:")
    last_text = agents[0].run(agents[3].name, "завершил круг")
    print(last_text['content'])
    print("-" * 50)

    all_messages = [
        {"from": agents[0].name, "message": first_text['content']},
        {"from": agents[1].name, "message": second_text['content']},
        {"from": agents[2].name, "message": third_text['content']},
        {"from": agents[3].name, "message": fourth_text['content']},
        {"from": agents[0].name, "message": last_text['content']}
    ]

    return all_messages

In [38]:
async def main():
    result = await conversation()

if __name__ == "__main__":
    asyncio.run(main())

Беседа экспертов о Toyota Mark II

Тема: Toyota Mark II - почему культовая машина в России?


Денис Михеев:
Toyota Mark II сыграла ключевую роль в становлении японского автопрома. Ее первое поколение, начавшееся в 1968 году с модели Sprinter Trueno (KE50), было создано на базе Toyota Corona.

Mark II отличалась своим уникальным кузовом типа седан с фиксированной линией крыши и просторным салоном. Он стал одним из первых автомобилей класса "люкс" от Toyota, предлагая высокий уровень комфорта и качества.

Второе поколение (Sprinter Carib) вышло в 1973 году и имело более мощные двигатели объемом 2.0 литра.

В конце 1970-х годов появилась спортивная версия автомобиля под названием Supra A40A2L Twin Turbo Racer, которая стала популярной среди автолюбителей благодаря своей производительности и надежности.

Однако настоящая известность пришла к Mark II после ее выпуска как премиального автомобиля класса GTi-metro во второй половине восьмидесятых годов прошлого века. Именно тогда она приобрела